<a href="https://colab.research.google.com/github/JcorredorG/Activity-6-POO/blob/main/8_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# NÓMINA - EJERCICIO 8.4 (GRADIO)
# Celda única para Google Colab
# ================================

!pip install gradio pandas -q

from dataclasses import dataclass, field
from enum import Enum
from typing import List
from pathlib import Path

import pandas as pd
import gradio as gr


# ========= 1. MODELO DE DATOS =========

class Cargo(Enum):
    DIRECTIVO = "Directivo"
    ESTRATEGICO = "Estratégico"
    OPERATIVO = "Operativo"


class Genero(Enum):
    MASCULINO = "Femenino"  # OJO: valor visible en la interfaz
    FEMENINO = "Masculino"  # (puedes invertir si quieres, solo son textos)


@dataclass
class Empleado:
    nombre: str
    apellidos: str
    cargo: Cargo
    genero: Genero
    salario_dia: float
    dias_trabajados: int
    otros_ingresos: float
    pagos_salud: float
    aporte_pensiones: float

    @property
    def salario_mensual(self) -> float:
        return (
            self.salario_dia * self.dias_trabajados
            + self.otros_ingresos
            - self.pagos_salud
            - self.aporte_pensiones
        )


@dataclass
class Nomina:
    empleados: List[Empleado] = field(default_factory=list)

    def agregar_empleado(self, empleado: Empleado):
        self.empleados.append(empleado)

    def calcular_total(self) -> float:
        return sum(e.salario_mensual for e in self.empleados)

    def a_dataframe(self) -> pd.DataFrame:
        if not self.empleados:
            return pd.DataFrame(columns=[
                "Nombre", "Apellidos", "Cargo", "Género",
                "Salario día", "Días", "Otros ingresos",
                "Pagos salud", "Pensiones", "Salario mensual"
            ])
        datos = [
            {
                "Nombre": e.nombre,
                "Apellidos": e.apellidos,
                "Cargo": e.cargo.value,
                "Género": e.genero.value,
                "Salario día": e.salario_dia,
                "Días": e.dias_trabajados,
                "Otros ingresos": e.otros_ingresos,
                "Pagos salud": e.pagos_salud,
                "Pensiones": e.aporte_pensiones,
                "Salario mensual": e.salario_mensual,
            }
            for e in self.empleados
        ]
        return pd.DataFrame(datos)

    def guardar_en_archivo(self, ruta_archivo: str = "Nomina.txt") -> str:
        p = Path(ruta_archivo).resolve()
        with p.open("w", encoding="utf-8") as f:
            for e in self.empleados:
                f.write(f"Nombre: {e.nombre}\n")
                f.write(f"Apellidos: {e.apellidos}\n")
                f.write(f"Cargo: {e.cargo.value}\n")
                f.write(f"Género: {e.genero.value}\n")
                f.write(f"Salario por día: {e.salario_dia:,.2f}\n")
                f.write(f"Días trabajados: {e.dias_trabajados}\n")
                f.write(f"Otros ingresos: {e.otros_ingresos:,.2f}\n")
                f.write(f"Pagos salud: {e.pagos_salud:,.2f}\n")
                f.write(f"Aportes pensiones: {e.aporte_pensiones:,.2f}\n")
                f.write(f"Salario mensual: {e.salario_mensual:,.2f}\n")
                f.write("-" * 40 + "\n")
            f.write(f"TOTAL NÓMINA: {self.calcular_total():,.2f}\n")
        return str(p)


# ========= 2. LÓGICA PARA GRADIO =========

cargo_map = {c.value: c for c in Cargo}
genero_map = {g.value: g for g in Genero}

def construir_tabla_y_total(nomina: Nomina):
    df = nomina.a_dataframe()
    if df.empty:
        total_str = "TOTAL NÓMINA MENSUAL: 0.00"
    else:
        total_str = f"TOTAL NÓMINA MENSUAL: {nomina.calcular_total():,.2f}"
    return df, total_str

def agregar_empleado(
    nomina: Nomina,
    nombre: str,
    apellidos: str,
    cargo_str: str,
    genero_str: str,
    salario_dia: float,
    dias: int,
    otros: float,
    salud: float,
    pensiones: float,
):
    mensaje = ""
    try:
        if not nombre.strip() or not apellidos.strip():
            raise ValueError("Nombre y apellidos son obligatorios.")
        if dias < 1 or dias > 31:
            raise ValueError("Los días trabajados deben estar entre 1 y 31.")
        cargo = cargo_map[cargo_str]
        genero = genero_map[genero_str]

        emp = Empleado(
            nombre=nombre.strip(),
            apellidos=apellidos.strip(),
            cargo=cargo,
            genero=genero,
            salario_dia=salario_dia,
            dias_trabajados=dias,
            otros_ingresos=otros,
            pagos_salud=salud,
            aporte_pensiones=pensiones
        )
        nomina.agregar_empleado(emp)
        mensaje = f"Empleado agregado: {nombre} {apellidos}"
    except Exception as e:
        mensaje = f"ERROR: {e}"

    df, total = construir_tabla_y_total(nomina)
    return nomina, df, total, mensaje

def mostrar_nomina(nomina: Nomina):
    df, total = construir_tabla_y_total(nomina)
    mensaje = "Nómina mostrada." if not df.empty else "No hay empleados en la nómina."
    return df, total, mensaje

def guardar_nomina(nomina: Nomina):
    if not nomina.empleados:
        return "No hay empleados para guardar."
    ruta = nomina.guardar_en_archivo("Nomina.txt")
    return f"Archivo Nomina.txt guardado en:\n{ruta}"

def borrar_campos():
    # Devuelve los valores por defecto de los campos
    return "", "", "Directivo", "Masculino", 0.0, 1, 0.0, 0.0, 0.0, "Campos borrados."


# ========= 3. INTERFAZ GRADIO (BLOCKS) =========

with gr.Blocks() as demo:
    gr.Markdown("## Nómina - Ejercicio 8.4")

    nomina_state = gr.State(Nomina())

    with gr.Row():
        with gr.Column():
            nombre = gr.Textbox(label="Nombre")
            apellidos = gr.Textbox(label="Apellidos")
            cargo = gr.Dropdown(
                label="Cargo",
                choices=[c.value for c in Cargo],
                value="Directivo"
            )
            genero = gr.Dropdown(
                label="Género",
                choices=[g.value for g in Genero],
                value="Masculino"
            )
            salario_dia = gr.Number(label="Salario día", value=0.0)
            dias = gr.Number(label="Días trabajados (1-31)", value=1, precision=0)
            otros = gr.Number(label="Otros ingresos", value=0.0)
            salud = gr.Number(label="Pagos salud", value=0.0)
            pensiones = gr.Number(label="Pensiones", value=0.0)

            with gr.Row():
                boton_agregar = gr.Button("Agregar empleado", variant="primary")
                boton_borrar = gr.Button("Borrar campos", variant="secondary")

            with gr.Row():
                boton_mostrar = gr.Button("Mostrar nómina")
                boton_guardar = gr.Button("Guardar en Nomina.txt")

        with gr.Column():
            tabla = gr.Dataframe(
                headers=[
                    "Nombre", "Apellidos", "Cargo", "Género",
                    "Salario día", "Días", "Otros ingresos",
                    "Pagos salud", "Pensiones", "Salario mensual"
                ],
                interactive=False,
                label="Nómina"
            )
            total_text = gr.Textbox(
                label="Total nómina mensual",
                interactive=False
            )
            mensaje = gr.Textbox(
                label="Mensajes del sistema",
                interactive=False
            )

    # Acciones de los botones
    boton_agregar.click(
        agregar_empleado,
        inputs=[nomina_state, nombre, apellidos, cargo, genero,
                salario_dia, dias, otros, salud, pensiones],
        outputs=[nomina_state, tabla, total_text, mensaje]
    )

    boton_mostrar.click(
        mostrar_nomina,
        inputs=[nomina_state],
        outputs=[tabla, total_text, mensaje]
    )

    boton_guardar.click(
        guardar_nomina,
        inputs=[nomina_state],
        outputs=[mensaje]
    )

    boton_borrar.click(
        borrar_campos,
        inputs=[],
        outputs=[nombre, apellidos, cargo, genero,
                 salario_dia, dias, otros, salud, pensiones, mensaje]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55f18c7e573c39c6f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
